In [2]:
## Instalar las librerías necesarias y leer un archivo para asignar el nombre de las bandas a proceso de subida a GEE
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Upload all bucket files in GC to GEE ASSET

In [13]:
import os
import json
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import google.auth

# Path to the JSON file with the credentials
credentials_path = 'path/to/credentials'
token_path = 'path/to/token.json' 

# Authentication using OAuth 2.0 client credentials
def get_credentials():
    credentials = None
    if os.path.exists(token_path):
        with open(token_path, 'r') as token_file:
            credentials = google.oauth2.credentials.Credentials.from_authorized_user_info(
                json.load(token_file)
            )

    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_path,
                scopes=['https://www.googleapis.com/auth/cloud-platform']
            )
            credentials = flow.run_local_server(port=0)

        # Guarda las credenciales para la próxima ejecución
        with open(token_path, 'w') as token_file:
            token_file.write(credentials.to_json())

    return credentials

credentials = get_credentials()


print("Authentication successful")

RefreshError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})

In [12]:
from google.oauth2 import service_account
from google.cloud import storage

# Reemplaza con la ruta a tu archivo de clave de cuenta de servicio
service_account_path = 'path/to/service/account'

# Configura las credenciales
credentials = service_account.Credentials.from_service_account_file(service_account_path)

# Inicializa el cliente de almacenamiento
storage_client = storage.Client(credentials=credentials)

# Lista los buckets
buckets = list(storage_client.list_buckets())
for bucket in buckets:
    print(bucket.name)


planet_tesis


In [9]:
import ee
ee.Authenticate()
ee.Initialize()



Successfully saved authorization token.


In [16]:
import os
import re
import pandas as pd
from google.oauth2 import service_account
from google.cloud import storage
import ee
from datetime import datetime

# Configure credentials file and parameters
service_account_path = 'path/to/service/account'
BUCKET_NAME = 'planet_tesis'
BASE_PATH = 'file_base_path'
COLLECTION = 'gee/project/collection'
BAND_NAMES = ['coastal_blue', 'blue', 'green_i', 'green', 'yellow', 'red', 'rededge', 'nir']
EXCEL_OUTPUT = 'gee_output_example.xlsx'  # Output file for uploaded files log

def initialize_ee():
    try:
        ee.Initialize()
        print("Google Earth Engine initialized.")
    except Exception as e:
        print(f"Error initializing Earth Engine: {e}")
        raise

def initialize_storage_client():
    creds = service_account.Credentials.from_service_account_file(service_account_path)
    return storage.Client(credentials=creds)

def list_files(bucket_name, base_path):
    print("Listing files in the bucket...")
    storage_client = initialize_storage_client()
    blobs = storage_client.list_blobs(bucket_name, prefix=base_path)
    # Only add files that match the exact pattern YYYY-MM-DD_strip_######_composite.tif
    pattern = r'\d{4}-\d{2}-\d{2}_strip_\d+_composite\.tif$'
    files = [blob.name for blob in blobs if re.search(pattern, blob.name)]
    print("Files listed:", files)
    return files

def extract_date_from_filename(filename):
    print(f"Extracting date from file: {filename}")
    match = re.search(r'(\d{4}-\d{2}-\d{2})', filename)  # Date in format YYYY-MM-DD
    if match:
        formatted_date = match.group(1)
        print(f"Date extracted: {formatted_date}")
        return formatted_date
    else:
        print(f"No date found in filename: {filename}")
        return None

def upload_file(uri_gcs, asset_name, formatted_date):
    print(f"Uploading file to GEE: {asset_name} with date {formatted_date}")
    asset_id = f"{COLLECTION}/{asset_name}"
    bands = [{'id': name, 'tileset_band_index': i} for i, name in enumerate(BAND_NAMES)]
    start_time = f"{formatted_date}T00:00:00Z"
    end_time = f"{formatted_date}T23:59:59Z"
    
    manifest = {
        'name': asset_id,
        'tilesets': [{
            'sources': [{
                'uris': [uri_gcs]
            }]
        }],
        'bands': bands,
        'startTime': start_time,
        'endTime': end_time
    }
    try:
        task_id = ee.data.newTaskId()[0]
        ee.data.startIngestion(task_id, manifest)
        print(f"Uploaded: {asset_name}")
        return True
    except Exception as e:
        print(f"Error uploading {asset_name}: {e}")
        return False

def main():
    initialize_ee()
    files = list_files(BUCKET_NAME, BASE_PATH)
    print("Files found:", files)
    
    uploaded_files = []  # To store data for Excel log

    for file in files:
        try:
            file_name = file.split('/')[-1]
            print("Processing file:", file_name)
            formatted_date = extract_date_from_filename(file_name)
            print("Extracted date:", formatted_date)
            if formatted_date:
                uri_gcs = f"gs://{BUCKET_NAME}/{file}"
                asset_name = file_name.split('.')[0]
                success = upload_file(uri_gcs, asset_name, formatted_date)
                # Add entry to the log if upload is successful
                if success:
                    uploaded_files.append({'File Name': file_name, 'Extracted Date': formatted_date})
        except Exception as e:
            print(f"Error processing file {file}: {e}")

    # Save the log to an Excel file
    df = pd.DataFrame(uploaded_files)
    df.to_excel(EXCEL_OUTPUT, index=False)
    print(f"Log saved to {EXCEL_OUTPUT}")

if __name__ == "__main__":
    main()


Google Earth Engine initialized.
Listing files in the bucket...
Files listed: ['Validacion_holm_oak/2023-05-01_strip_6474673_composite.tif', 'Validacion_holm_oak/2023-05-07_strip_6489539_composite.tif', 'Validacion_holm_oak/2023-05-07_strip_6489944_composite.tif', 'Validacion_holm_oak/2023-05-14_strip_6507234_composite.tif', 'Validacion_holm_oak/2023-05-14_strip_6508079_composite.tif', 'Validacion_holm_oak/2023-05-16_strip_6512391_composite.tif', 'Validacion_holm_oak/2023-05-19_strip_6519086_composite.tif', 'Validacion_holm_oak/2023-05-19_strip_6519238_composite.tif', 'Validacion_holm_oak/2023-06-04_strip_6555756_composite.tif', 'Validacion_holm_oak/2023-06-06_strip_6560471_composite.tif', 'Validacion_holm_oak/2023-06-17_strip_6584624_composite.tif', 'Validacion_holm_oak/2023-06-17_strip_6584725_composite.tif', 'Validacion_holm_oak/2023-06-24_strip_6600702_composite.tif', 'Validacion_holm_oak/2023-06-29_strip_6612834_composite.tif', 'Validacion_holm_oak/2023-06-30_strip_6614460_composi